## Análise de Indicadores

In [2]:
import pandas as pd
import pronouncing
import math
from scipy import stats

### Leitura das amostras

In [3]:
pw = pd.read_csv("./samples.csv", sep=';')

In [4]:
pw.head()

,Password,type,entropy
0,n(#?wb%874_^1^P@#)1,Random Passwords,3.9441
1,0WMuGYZqCm&YPFbMt,Random Passwords,3.8594
2,b(E?$3~+598w)m5-#,Random Passwords,3.9774
3,tD$)%H?(+#X#v-@6o,Random Passwords,3.9774
4,5GVhzWToSrh~pAWzkAa,Random Passwords,3.8363


### Indicadores

In [5]:
'''
Função para contar a quantidade de sílabas em cada senha, quando aplicável
'''
def syllable_count(password, sep='-'):
    pronunciations = []
    words = password.split(sep)
    syllable_c = 0
    for w in words:
        pronunciation_list = pronouncing.phones_for_word(w)
        if (pronunciation_list):
            syllable_c += pronouncing.syllable_count(pronunciation_list[0])
        else:
            syllable_c += 0
    return syllable_c

'''
Função para medir a pronuncia de cada senha, quando aplicável
'''
def get_pronunciation(password, sep='-'):
    pronunciations = []
    words = password.split(sep)
    for w in words:
        pronunciation = 0
        phones_list = pronouncing.phones_for_word(w)
        if (phones_list):
            p = pronouncing.stresses(phones_list[0])
            for i in p:
                if i == '0':
                    pronunciation+=1
                elif i == '1':
                    pronunciation+=3
                elif i == '2':
                    pronunciation+=2
            pronunciations.append(pronunciation)
        else:
             pronunciations.append(0)
    return sum(pronunciations) / len(pronunciations)

'''
Função em python para medir a entropia de cada senha
'''
def sci_entropy(password):
    character_set = set(password)
    password_length = len(password)
    probability_distribution = pd.Series(list(password.count(char) / password_length for char in character_set), index=character_set)
    entropy = stats.entropy(probability_distribution, base=2)
    return entropy

#### Quantidade de caracteres

In [6]:
pw["length"] = pw['Password'].apply(lambda x : len(x))

#### Quantidade de caracteres únicos

In [7]:
pw["char_set"] =  pw['Password'].apply(lambda x : len(set(x)))

#### Quantidade de sílabas

In [8]:
pw["syllable_count"] = pw['Password'].apply(lambda x : syllable_count(x))

#### Indicador de Pronunciabilidade

In [9]:
pw["pronouncing"] = pw['Password'].apply(lambda x : get_pronunciation(x))

#### Entropia

In [10]:
pw['entropy_sci'] = pw['Password'].apply(sci_entropy)

### Tabelas

### Média dos indicadores para cada tipo de senha

In [12]:
pw.groupby(by='type').mean()

,entropy,length,char_set,syllable_count,pronouncing,entropy_sci
type,,,,,,
Dict Passwords,2.320790,5.593,5.299,1.705,3.7730,2.320353
Munged Passwords,2.270266,5.571,5.148,0.042,0.1200,2.269841
Passphrases,3.444087,18.756,12.064,5.068,3.7580,3.441478
Random Passwords,3.849219,18.467,15.276,0.037,0.0431,3.840923


### Correlação entre os indicadores - agrupado por tipo

In [13]:
pw.groupby(by='type').corr()

entropy    length  char_set  syllable_count  \
type                                                                            
Dict Passwords   entropy         1.000000  0.905829  0.976167        0.682510   
                 length          0.905829  1.000000  0.952629        0.798746   
                 char_set        0.976167  0.952629  1.000000        0.746038   
                 syllable_count  0.682510  0.798746  0.746038        1.000000   
                 pronouncing     0.660276  0.774249  0.726535        0.965132   
                 entropy_sci     0.999998  0.905977  0.976304        0.682541   
Munged Passwords entropy         1.000000  0.884299  0.974777       -0.154075   
                 length          0.884299  1.000000  0.939386       -0.161560   
                 char_set        0.974777  0.939386  1.000000       -0.153519   
                 syllable_count -0.154075 -0.161560 -0.153519        1.000000   
                 pronouncing    -0.162017 -0.169429 -0.160438        0.987366   
                 entropy_sci     0.999998  0.884424  0.974873       -0.154017   
Passphrases      entropy         1.000000  0.496752  0.963167        0.396063   
                 length          0.496752  1.000000  0.629244        0.824011   
                 char_set        0.963167  0.629244  1.000000        0.514529   
                 syllable_count  0.396063  0.824011  0.514529        1.000000   
                 pronouncing     0.380430  0.810443  0.496164        0.961080   
                 entropy_sci     0.999553  0.507339  0.965387        0.405795   
Random Passwords entropy         1.000000  0.352650  0.981715        0.006950   
                 length          0.352650  1.000000  0.471010       -0.084253   
                 char_set        0.981715  0.471010  1.000000       -0.003108   
                 syllable_count  0.006950 -0.084253 -0.003108        1.000000   
                 pronouncing     0.032540 -0.063189  0.021478        0.948714   
                 entropy_sci     0.999528  0.366128  0.983732        0.006489   

                                 pronouncing  entropy_sci  
type                                                       
Dict Passwords   entropy            0.660276     0.999998  
                 length             0.774249     0.905977  
                 char_set           0.726535     0.976304  
                 syllable_count     0.965132     0.682541  
                 pronouncing        1.000000     0.660351  
                 entropy_sci        0.660351     1.000000  
Munged Passwords entropy           -0.162017     0.999998  
                 length            -0.169429     0.884424  
                 char_set          -0.160438     0.974873  
                 syllable_count     0.987366    -0.154017  
                 pronouncing        1.000000    -0.161950  
                 entropy_sci       -0.161950     1.000000  
Passphrases      entropy            0.380430     0.999553  
                 length             0.810443     0.507339  
                 char_set           0.496164     0.965387  
                 syllable_count     0.961080     0.405795  
                 pronouncing        1.000000     0.390540  
                 entropy_sci        0.390540     1.000000  
Random Passwords entropy            0.032540     0.999528  
                 length            -0.063189     0.366128  
                 char_set           0.021478     0.983732  
                 syllable_count     0.948714     0.006489  
                 pronouncing        1.000000     0.031760  
                 entropy_sci        0.031760     1.000000